# Eat Safe, Love

## Notebook Set Up

In [15]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [16]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [17]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [18]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [19]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [20]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene' : 20}

# Use count_documents to display the number of documents in the result
hygiene_score = establishments.count_documents(query)
print(f'There are {hygiene_score} restaurants with a hygiene score of 20.')
# Display the first document in the results using pprint
pprint(establishments.find_one(query))

There are 0 restaurants with a hygiene score of 20.
None


In [22]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
hygiene_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
number_of_rows = hygiene_df['_id'].count()
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

KeyError: '_id'

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [23]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex' : 'London'} , 'RatingValue': {'$gte' : '4'}}

# Use count_documents to display the number of documents in the result
rating_value = establishments.count_documents(query)
print(f'There are {rating_value} restaurants with a rating value greater than or equal to 4.')

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

There are 0 restaurants with a rating value greater than or equal to 4.
None


In [24]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
rating_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
number_of_rows = rating_df['_id'].count()
print(f'There are {number_of_rows} rows in the dataframe.')
# Display the first 10 rows of the DataFrame
rating_df.head(10)

KeyError: '_id'

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [25]:
fields = {'geocode': 1}

establishments.find_one({'geocode.latitude': {'$gte' : 5.5}} ,fields)

{'_id': ObjectId('64d82e799a15b3b69c2c22cf'),
 'geocode': {'longitude': Decimal128('0.08384000'),
  'latitude': Decimal128('51.49014200')}}

In [26]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.083812
longitude = 1.195625

query = {'RatingValue': '5' , 
         '$and': [
            {'geocode.longitude' : {'$gte' : longitude - degree_search}} , {'geocode.longitude' : {'$lte' :longitude + degree_search}},
            { 'geocode.latitude' : {'$gte' : latitude - degree_search}} ,{'geocode.latitude' : {'$lte' : latitude + degree_search}}
sort =  [('scores.Hygiene' , 1)]
limit = 5
# Print the results
pprint(list(establishments.find(query).sort(sort).limit(limit)))

SyntaxError: invalid syntax (1693836740.py, line 13)

In [27]:
# Convert result to Pandas DataFrame
results = list(establishments.find(query).sort(sort).limit(limit))

top_5_df = pd.DataFrame(results)

top_5_df

NameError: name 'sort' is not defined

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [28]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match' : {'scores.Hygiene' : 0}}
# 2. Groups the matches by Local Authority
group_query = {'$group' :{ '_id' : '$LocalAuthorityName' , 'count' : {'$sum' : 1}}}
# 3. Sorts the matches from highest to lowest
sort_query = {'$sort' : {'count' : -1}}
pipeline = [match_query , group_query , sort_query]

results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print(f'The number of establishments with a higiene score of 0 is: {len(results)}.')
# Print the first 10 results
pprint(results[0:10])

The number of establishments with a higiene score of 0 is: 0.
[]


In [29]:
# Convert the result to a Pandas DataFrame
hygiene_score_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
number_of_rows = hygiene_score_df['_id'].count()
print(f'There are {number_of_rows} rows in the dataframe.')

# Display the first 10 rows of the DataFrame
hygiene_score_df.head(10)

KeyError: '_id'